In [1]:
import quippy

In [2]:
from quippy.descriptors import Descriptor

In [3]:
import numpy as np
import pandas as pd
import ase
import os

#from quippy.descriptors import Descriptor
#import quippy.descriptors.Descriptor as Descriptor
from quippy.descriptors import Descriptor
from ase import Atoms

at = ase.Atoms(
        symbols=['O', 'Ga'],
               )
at.set_cell

at.set_scaled_positions

desc = Descriptor(
        "soap cutoff=10 l_max=4 n_max=4 normalize=F atom_sigma=0.5 n_Z=4 Z={8 13 31 49} n_species=4 species_Z={8 13 31 49}" # n_Z=1 Z={14} 
        )

def get_path(tt: str):
    path = '/home/jurka/research-project/'+tt+'/final/'
    return path


### Get the train/test data

In [4]:
df_xyz_train = pd.read_csv('/root/atoms_xyz_train.csv')
print(df_xyz_train)

df_xyz_test = pd.read_csv('/root/atoms_xyz_test.csv')
print(df_xyz_test)

df_energy_train = pd.read_csv('/root/energy_train.csv')
print(df_energy_train)

df_energy_test = pd.read_csv('/root/energy_test.csv')
print(df_energy_test)

df_frac_xyz_train = pd.read_csv('/root/atoms_frac_xyz_train.csv')
print(df_frac_xyz_train)

df_frac_xyz_test = pd.read_csv('/root/atoms_frac_xyz_test.csv')
print(df_frac_xyz_test)

df_lattice_train = pd.read_csv('/root/lattice_vector_train.csv')
print(df_lattice_train)

df_lattice_test = pd.read_csv('/root/lattice_vector_test.csv')
print(df_lattice_test)

          id species     x [A]     y [A]     z [A]
0          1      Ga  1.630872  7.250740  6.324361
1          1      Al  6.630021  7.245745  6.336774
2          1      Al  3.386863  1.289255  1.763723
3          1      Ga  8.308420  1.304978  1.782694
4          1      Ga  0.909800  2.994510  1.764503
...      ...     ...       ...       ...       ...
147445  2400       O  5.975104  8.099874  8.040530
147446  2400       O  1.018043  7.646383  8.016728
147447  2400       O  5.476533  6.034025  1.000423
147448  2400       O  8.194732  3.099933  5.646403
147449  2400       O  8.105981  6.051474  8.058389

[147450 rows x 5 columns]
        id species     x [A]     y [A]     z [A]
0        1      In  1.342078  7.643380  6.403149
1        1      In  6.634789  7.545168  6.217297
2        1      In  3.835553  1.275290  1.727485
3        1      In  9.018650  1.170920  1.575733
4        1      In  1.244861  3.332692  1.979092
...    ...     ...       ...       ...       ...
37035  600       O

## ASE Testing with some deprecated soap testing

In [22]:
atoms = Atoms('N3', [(0, 0, 0), (1, 0, 0), (0, 0, 1)])

In [23]:
atoms.get_positions()

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [24]:
atoms.set_positions([(2, 0, 0), (1, 1, 0), (2, 2, 0)])

In [25]:
atoms.get_positions()

array([[2., 0., 0.],
       [1., 1., 0.],
       [2., 2., 0.]])

In [26]:
atoms.get_atomic_numbers()

array([7, 7, 7])

In [27]:
atoms.set_atomic_numbers([5,5,5])

In [28]:
atoms.get_atomic_numbers()

array([5, 5, 5])

In [105]:
from ase.build import molecule
atoms = molecule('CH3CH2OH')
#del atoms[[atom.index for atom in atoms if atom.symbol=='H']]

In [106]:
atoms
# n_Z=2 Z={1 6} n_species=2 species_Z={1 6}

Atoms(symbols='C2OH6', pbc=False)

In [107]:
SOAP_descriptor = desc.calc(atoms)['data']#.flatten()

In [108]:
desc.calc_descriptor(atoms).shape

(9, 51)

In [109]:
(SOAP_descriptor == desc.calc_descriptor(atoms)).all()

True

## Test on our data with default settings
Should the values of n_Z and n_species together with the sets change when the actual number of species in the compound changes?

In [175]:
# Test on our data if we get 681 long descriptor:
structure = df_xyz_train[df_xyz_train.id==1]
atoms = Atoms(structure.species.to_list(), structure[structure.columns[-3:]].to_numpy())
desc = Descriptor(
        "soap cutoff=10 l_max=4 n_max=4 normalize=T atom_sigma=0.5 n_Z=4 Z={8 13 31 49} n_species=4 species_Z={8 13 31 49}" # n_Z=1 Z={14} 
        )

In [182]:
mean_soap = np.mean(soap, axis=0)

## Calculate the descriptors for the Kaggle competition data

In [52]:
def calc_desc(df_xyz):
    df = {}
    desc = Descriptor(
        "soap cutoff=10 l_max=2 n_max=3 normalize=F atom_sigma=0.5 n_Z=4 Z={8 13 31 49} n_species=4 species_Z={8 13 31 49}" # n_Z=1 Z={14} 
        )
    for i in df_xyz.id.unique():
        struct = df_xyz[df_xyz.id==i]
        atoms = Atoms(struct.species.to_list(), struct[struct.columns[-3:]].to_numpy())
        mean_soap = np.mean(desc.calc_descriptor(atoms), axis=0)
        df[i] = mean_soap
    return pd.DataFrame(df).transpose().to_numpy()

In [53]:
# je sigma i pro stredovy atom?
# github issues zeptat se
# je soap komutativni pres okoli?

In [54]:
soap_train = calc_desc(df_xyz_train)
soap_test = calc_desc(df_xyz_test)

In [56]:
for i in soap_train:
    print(i[0])

0.0044178248638169545
0.004415490277537292
0.004415695500712793
0.004412094458332642
0.004411407961760184
0.004414537475627204
0.0044089386052379095
0.0044127243697104305
0.004412342662320412
0.004414879914579517
0.004413799715247077
0.0044156781405760934
0.0044171839073331885
0.004412490694586154
0.004412768836965622
0.004414522276943755
0.004417526831568399
0.004411241630034262
0.004411029650138067
0.004411567965161114
0.004411289946914741
0.004412818423384439
0.004411789979314756
0.0044122363737014855
0.004413162880390215
0.004416916540781875
0.004412502802690682
0.004416694953314671
0.0044120289784478515
0.004414133656665405
0.004413095392462431
0.004409460449901993
0.004412353779962673
0.0044161045652684324
0.004410272851064017
0.004414518992999429
0.004413873897383559
0.004411668518125325
0.0044143578549099335
0.004411206602455019
0.004413154902669604
0.0044175796314359465
0.004414248447818338
0.0044175570590433725
0.004412064507470457
0.004416601839230161
0.004415225588109588
0.

0.004412542946191298
0.004414848179267219
0.004414420759774309
0.004417495400731629
0.004413074773084999
0.004412667416766139
0.004414907276650516
0.004414566245286145
0.004416167363984861
0.004413703906451677
0.00441190592060644
0.004413075044173797
0.004411686613170036
0.004413244104825663
0.004411855167366084
0.004413818012922173
0.004414017532765523
0.004410899667150799
0.004412394062876813
0.004416487186252569
0.0044126718963529285
0.0044126194809225115
0.004412521271073052
0.004411322777020085
0.004411656604079464
0.0044156214304079565
0.00441382789935892
0.0044144263672860765
0.004412367351894652
0.004412269973656687
0.0044167334020965305
0.004411021791863241
0.0044111977326603165
0.004412963754734194
0.0044123898228167376
0.004415301277308356
0.004409252047012024
0.004415058148662834
0.004418456210969228
0.004416946694642316
0.004414287791707295
0.004411850897228909
0.004410084572837077
0.004413316326712
0.004410758275688829
0.004409781783056014
0.004412081387684215
0.004412677

In [41]:
count=0
for i in range(1, 4*3+1):
    for j in range(i,4*3+1):
        for l in range(2+1):
            count+=1
print(count)

234


In [19]:
soap_train

array([[2.64466713e-03, 3.10027300e-03, 4.12572445e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.64466687e-03, 2.23541286e-03, 2.03508434e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.64465723e-03, 1.53104204e-03, 7.59355208e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.64468109e-03, 3.03651548e-03, 3.88451426e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.64465212e-03, 2.38880312e-03, 2.99105666e-03, ...,
        1.60352137e-04, 8.11598572e-05, 0.00000000e+00],
       [2.64469472e-03, 3.07099513e-03, 3.89714793e-03, ...,
        9.96596965e-15, 9.70651902e-08, 0.00000000e+00]])

In [16]:
np.savetxt('/root/soap_train_unnormalized.csv', soap_train, delimiter=",")
np.savetxt('/root/soap_test_unnormalized.csv', soap_test, delimiter=",")

In [17]:
# soap_*_unnormalized -- not normalized, normalize=F
# soap_*_default -- normalized, normalize=T